## Fuzzy matching 

input files: pv_disc_26Apr20.csv,crsp_disc_26Apr20.csv
output files: fuzzymatched_crsp_pv_30Apr20.csv

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import time
import numpy as np

In [2]:
ctm = pd.read_csv("pv_disc_26Apr20.csv")
pv3 = pd.read_csv("crsp_disc_26Apr20.csv")

In [3]:
pv_letters = list(pv3['org_first'].unique())

In [4]:
## fuzzy calculation for standardized clean names

## use threshold score as 70

dfc = pd.DataFrame({"crsp": ctm.comlow4.unique()})
pv_letters = list(pv3.org_first.unique())
pv_letters.reverse()
for i in pv_letters:
    print(i)
    A = time.time()
    
    dfp = pd.DataFrame({"pv": pv3[pv3['org_first']==i]['orglow9'].unique()})
    x = np.array(np.meshgrid(dfp.pv.values, dfc.crsp.values)).T.reshape(-1,2) # create matrix for pairwise fuzzy score
    df3 = pd.DataFrame(x)
    
    df3.columns = ['pv','crsp']
    
    df3['Ratio'] = [fuzz.ratio(*i) for i in map(tuple, x)]
    df3['Token'] = [fuzz.token_set_ratio(*i) for i in map(tuple, x)]
    print(time.time()-A)
    df3 = df3[(df3['Ratio']>70) | (df3['Token']>70)]
    name = "sub_"+i+".csv"
    df3.to_csv(name, index = False)

In [5]:

## fuzzy calculation for new clean names (after removing high frequency last words)

## use threshold score as 70

dfc = pd.DataFrame({"crsp_new": ctm.comlow_new.unique()})

pv_letters.reverse()
for i in pv_letters:
    print(i)
    A = time.time()
    
    dfp = pd.DataFrame({"pv_new": pv3[pv3['org_first']==i]['orglow_new'].unique()})
    x = np.array(np.meshgrid(dfp.pv_new.values, dfc.crsp_new.values)).T.reshape(-1,2)
    df3 = pd.DataFrame(x)
    
    df3.columns = ['pv_new','crsp_new']
    
    df3['new_ratio'] = [fuzz.ratio(*i) for i in map(tuple, x)]
    df3['new_token'] = [fuzz.token_set_ratio(*i) for i in map(tuple, x)]
    df3['new_fpr'] = [fuzz.partial_ratio(*i) for i in map(tuple, x)]
    
    print(time.time()-A)
    df3 = df3[(df3['new_ratio']>70) | (df3['new_token']>70) | (df3['new_fpr']>70)]
    name = "sub_new"+i+".csv"
    df3.to_csv(name, index = False)

###### from fuzzy matrix input, filter matches based on the following rules:

1. for a given pair, if max(fuzzy_ratio, fuzzy_token) == 100, retain the pair
2. If there is more than one match for a company name which has max(fuzzy_ratio, fuzzy_token) == 100, then retain the match with higher fuzz_ratio 
3. for pairs where max(fuzzy_ratio, fuzzy_token) != 100:
    - retain all the names which have only one single match
    - for names which have more than one match, retain the match with higher max(fuzzy_ratio, fuzzy_token)
    - for names which have more than one match with same max(fuzzy_ratio, fuzzy_token), use higher ratio to break the tie
4. finally of all the retained pairs, select only those with max(fuzzy_ratio, fuzzy_token) >70
    


In [7]:
for i in pv_letters:

    ####for clean company names after removing high frequency last words 
    name = "sub_new"+i+".csv"
    newe = pd.read_csv(name)
    newe['mx'] = newe.apply(lambda x: max(x.new_ratio, x.new_token), axis = 1)


    newe100 = newe[newe['mx']==100]  #for a given pair, if max(fuzzy_ratio, fuzzy_token) == 100, retain the pair
    newe12 = newe100.drop_duplicates('pv_new', keep = False) ##retain- add to a list at the end
    newe1 = newe100[newe100.duplicated('pv_new', keep = False)].reset_index() ##if tie, use ratio score to break it

    newe2 =  newe1.loc[newe1.groupby(['pv_new'])['mx'].idxmax()]
    idx = newe2.groupby(['pv_new'], sort=False)['new_ratio'].transform(max) == newe2['new_ratio']
    newe3 = newe2[idx].sort_values(['pv_new']) ##to add at the end

    newe33 = newe1[newe1.duplicated(['pv_new','new_ratio','new_token'], keep = False)] ## remove the matches already retained



    ###not 100
    newetemp = newe[~(newe['mx']==100)]
    newe123 = newetemp.drop_duplicates('pv_new', keep = False) ## retain all the names which have only one single match to add to list at the end
    newe1t = newetemp[newetemp.duplicated('pv_new', keep = False)] # remove matches already retained

    new2 =  newe1t.loc[newe1t.groupby(['pv_new'])['mx'].idxmax()] # for names which have more than one match, retain the match with higher max(fuzzy_ratio, fuzzy_token)
    idx2 = new2.groupby(['pv_new'], sort=False)['mx'].transform(max) == new2['mx']
    new3 = new2[idx2].sort_values(['pv_new']) ### contains some duplicates in pv

    newe4 =  new3.loc[new3.groupby(['pv_new'])['new_ratio'].idxmax()] ## use ratio to break tie
    idx3 = newe4.groupby(['pv_new'], sort=False)['new_ratio'].transform(max) == newe4['new_ratio']
    new5 = newe4[idx3].sort_values('pv_new') ##add

    new5 = new5.append(newe12)
    new6 = new5.append(newe3)
    new7 = new6.append(newe123)
    new8 = new7.append(newe33)
    new9 = new8[new8['mx']>70]

    new9 = new9.drop_duplicates(keep = 'first')
    
    ######################################################
    
     ####for clean company names after standardizing
    lname = "sub_"+i+".csv"
    lowe = pd.read_csv(lname)
    lowe['mx'] = lowe.apply(lambda x: max(x.Ratio, x.Token), axis = 1)

    ##100
    
    lowe100 = lowe[lowe['mx']==100]
    lowe12 = lowe100.drop_duplicates('pv', keep = False) ##to add at the end
    lowe1 = lowe100[lowe100.duplicated('pv', keep = False)].reset_index() ##duplicates based on same pv

    lowe33 = lowe1[lowe1.duplicated(['pv','Ratio','Token'], keep = False)]




    lowe2 =  lowe1.loc[lowe1.groupby(['pv'])['mx'].idxmax()]
    idx = lowe2.groupby(['pv'], sort=False)['Ratio'].transform(max) == lowe2['Ratio']
    lowe3 = lowe2[idx].sort_values(['pv']) ##to add at the end

    ###not 100

    lowetemp = lowe[~(lowe['mx']==100)]
    lowe123 = lowetemp.drop_duplicates('pv', keep = False) ##to add at the end
    lowe1t = lowetemp[lowetemp.duplicated('pv', keep = False)]

    low2 =  lowe1t.loc[lowe1t.groupby(['pv'])['mx'].idxmax()]
    idx2 = low2.groupby(['pv'], sort=False)['mx'].transform(max) == low2['mx']
    low3 = low2[idx2].sort_values(['pv'])  #of these some have multiple

    lowe4 =  low3.loc[low3.groupby(['pv'])['Ratio'].idxmax()]
    idx3 = lowe4.groupby(['pv'], sort=False)['Ratio'].transform(max) == lowe4['Ratio']
    low5 = lowe4[idx3].sort_values('pv') ##add
    
    ##if duplicates, retain one with same starting string


    low5 = low5.append(lowe12)
    low6 = low5.append(lowe3)
    low7 = low6.append(lowe123)
    low8 = low7.append(lowe33)
    low9 = low8[low8['mx']>70]

    low9 = low9.drop_duplicates(keep = 'first')

#####################################################################
    ####join with crsp##
    df_new =  pd.merge(new9, ctm, how = "left",left_on = ['crsp_new'], right_on = ['comlow_new'])
    df_low =  pd.merge(low9, ctm, how = "left",left_on = ['crsp'], right_on = ['comlow4'])

    ##remove duplicates###
    df_new = df_new.drop_duplicates(keep = 'first')
    df_low = df_low.drop_duplicates(keep = 'first')
    
    ###merge with pv###
    
    pv_sub = pv3[pv3['org_first']==i]
    pv_sub = pv_sub[['assignee_id', 'organization', 'orglow9','orglow_new', 'org_first']]
    pv_sub = pv_sub.drop_duplicates(keep = 'first')
    
    
    pc_new = pd.merge(pv_sub,df_new, left_on = ['orglow_new'], right_on = ['pv_new'], how = "left")
    pc_low = pd.merge(pv_sub,df_low,  left_on = ['orglow9'], right_on = ['pv'], how = "left")
    
    
    # remove na
    pc_new = pc_new[~pc_new['pv_new'].isna()]
    pc_low = pc_low[~pc_low['pv'].isna()]
    
    pc_new2 = pc_new[['assignee_id', 'organization', 'orglow9', 'orglow_new',
       'pv_new', 'COMNAM','crsp_new',  'comlow4', 'comlow_new',
                  'new_ratio', 'new_token', 'new_fpr','PERMNO']]
    pc_low2 = pc_low[['assignee_id', 'organization', 'orglow9', 'orglow_new','pv', 'crsp', 'Ratio', 'Token',
                 'PERMNO', 'COMNAM','comlow4','comlow_new']]
    
    ###join as one data frame

    pc = pd.merge(pc_new2, pc_low2, how = "outer",  
              left_on =['assignee_id', 'organization', 'orglow9', 'orglow_new', 'COMNAM','PERMNO',
                       'comlow4','comlow_new'],
             right_on = ['assignee_id', 'organization', 'orglow9', 'orglow_new', 'COMNAM','PERMNO',
                        'comlow4','comlow_new'])
    
    pc['Ratio'] = pc.apply(lambda x: fuzz.ratio(x.comlow4, x.orglow9), axis = 1)
    pc['Token'] = pc.apply(lambda x: fuzz.token_set_ratio(x.comlow4, x.orglow9), axis = 1)
    pc['new_ratio'] = pc.apply(lambda x: fuzz.ratio(x.comlow_new, x.orglow_new), axis = 1)
    pc['new_token'] = pc.apply(lambda x: fuzz.token_set_ratio(x.comlow_new, x.orglow_new), axis = 1)
    pc['new_fpr'] = pc.apply(lambda x: fuzz.partial_ratio(x.comlow_new, x.orglow_new), axis = 1)
    
    

In [15]:
# compute max of top two scores. after trial and error,

## after testing variations of  average score, linear combination of various scores, I figured that this
# measure gives good results
## and can be used at varying threshold to calculate precision and recall to get good matches

pc['top2'] = pc.apply(lambda x: statistics.mean(sorted([x.new_ratio,x.new_token,x.Token,x.Ratio])[2:]),axis = 1)

In [16]:
##compute average of fuzzy scores after removing high freq last words

pc['avg_low'] = pc.apply(lambda x: statistics.mean(([x.new_ratio,x.new_token])),axis = 1)

In [17]:
##compute average of fuzzy scores after standardizing and cleaning names

pc['avg_new'] = pc.apply(lambda x: statistics.mean(([x.Ratio,x.Token])),axis = 1)

In [18]:
# average of all scores
pc['avg'] = pc.apply(lambda x: statistics.mean(sorted([x.new_ratio,x.new_token,x.Token,x.Ratio])),axis = 1)

In [11]:
## use manual testing to check if proper match or not##
## create "finalmerge" column to mark if match == True or False

In [14]:
pc.to_csv("fuzzymatched_crsp_pv_30Apr20.csv", index = False)

